# Coursera Capstone Project
### This notebook will be use to complete the final project of the IBM Data Science Professional Certificate

# Step 1: Set up Notebook

In [2]:
print("HELLO CAPSTONE PROJECT COURSE")

HELLO CAPSTONE PROJECT COURSE


In [1]:
import pandas as pd
import numpy as np

import pandas as pd # library for data analsysis

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         237 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

In [4]:
! pip install beautifulsoup4

     |████████████████████████████████| 102kB 10.9MB/s ta 0:00:01


In [5]:
! pip install lxml

     |████████████████████████████████| 5.8MB 30.5MB/s eta 0:00:01


In [6]:
! pip install request

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/30/84/5f/484cfba678967ef58c16fce6890925d5c7172622f20111fbfd
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c1/e3/c1/d02c8c58538853e4c9b78cadb74f6d5c5c370b48a69a7271aa
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c3/c3/24/b5c132b537ab380c02d69e6bd4dec1f5db56b5fe19030473d7
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/d6/a4/78/01b20a9dc224dcc009fab669f7f27b943b8889c5150bd68d8a
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/23/7c/6e/f5b4e09d6596c8b8802b347e48f149031e2363368048f1347a
Successfully built request get post query-string public


In [7]:
from bs4 import BeautifulSoup as BS
import requests as r

# Step 2: Retrieve the data from Wikipedia

#### Here, Beautiful Soup is used to find the needed table from Wikipedia and placed in a temporary dataframe called 'n_list'

In [8]:
res = r.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BS(res.content,'lxml')
table = soup.find_all('table')[0]
n_list = pd.read_html(str(table))
n_list=n_list[0]
n_list.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### We remove all 'Not assigned' boroughs

In [9]:
n_list.set_index('Borough', inplace = True)
n_list.drop("Not assigned",axis=0, inplace = True)
n_list.reset_index(inplace=True)
n_list.set_index('Postcode', inplace = True)
n_list.reset_index(inplace=True)
n_list.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### The dataframe is now grouped based on the Postcode, combining the Neighbourhoods accordingly. We use commas to separate them.

In [10]:
n_list.sort_values("Postcode", ascending = True, inplace = True)
n_df=n_list.set_index('Postcode').groupby(level=['Postcode'],sort=False).agg(','.join)
n_df=n_df.reset_index()
n_dictionary=n_list[['Postcode','Borough']].set_index('Postcode').to_dict()
n_df['Borough'] = n_df.Postcode.map(n_dictionary['Borough'])
n_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Step 3: Clean the data

#### There is one "Not assigned" neighbourhood which will be replaced by the name of its borough.

In [11]:
n_df[n_df.Neighbourhood=='Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [12]:
# because it's only one, we can complete this by simply replacing the value manually
n_df['Neighbourhood']=n_df.Neighbourhood.str.replace("Not assigned", "Queen's Park")

In [13]:
# We can check this record using its index (85)
n_df.iloc[[85]]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


#### Let's check our total number of rows

In [15]:
print("The number of rows in this dataframe is",n_df.shape[0])

The number of rows in this dataframe is 103


#### Moving on, we can use the csv file, "Geospatial_data" provided by Coursera to create a dataframe of the Postal codes and their coordinates.

In [16]:
url = 'http://cocl.us/Geospatial_data'
geodata = pd.read_csv(url)
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### It's helpful if the naming conventions for our dataframes be consistant (i.e. "Postcode"; not "Postal Code")

In [17]:
geodata.rename(columns={'Postal Code': 'Postcode'},inplace=True)
geodata.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
geolat_dictionary=geodata[['Postcode','Latitude']].set_index('Postcode').to_dict()
geolat_dictionary=geolat_dictionary['Latitude']
geolong_dictionary=geodata[['Postcode','Longitude']].set_index('Postcode').to_dict()
geolong_dictionary=geolong_dictionary['Longitude']
n_df['Latitude'] = n_df.Postcode.map(geolat_dictionary)
n_df['Longitude'] = n_df.Postcode.map(geolong_dictionary)
n_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Richview Gardens,Kingsview Village,St. Phillip...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
